In [ ]:
%%writefile median.cpp
#include <opencv2/opencv.hpp>
#include <iostream>
#include <vector>
#include <filesystem>
#include <chrono>
#include <string>
#include <sys/resource.h>
#include <sys/times.h>
#include <unistd.h>
#include <cmath>

namespace fs = std::filesystem;
using namespace std;
using namespace cv;

Mat apply_median_filter(const Mat& image, int kernel_size) {
    Mat filtered_image;
    medianBlur(image, filtered_image, kernel_size);
    return filtered_image;
}

double compute_psnr(const Mat& image, const Mat& filtered_image) {
    Mat s1;
    absdiff(image, filtered_image, s1);
    s1.convertTo(s1, CV_32F);
    s1 = s1.mul(s1);
    Scalar s = sum(s1);
    double mse = s.val[0] / (double)(image.channels() * image.total());
    if (mse == 0) return 100;
    double psnr = 10.0 * log10((255 * 255) / mse);
    return psnr;
}

tuple<double, double, double> measure_metrics(const Mat& image, const Mat& filtered_image, double inference_time, const struct tms& start_time, const struct tms& end_time) {
    double psnr_value = compute_psnr(image, filtered_image);

    clock_t ticks_per_second = sysconf(_SC_CLK_TCK);
    double user_cpu_time = (end_time.tms_utime - start_time.tms_utime) / (double) ticks_per_second;
    double sys_cpu_time = (end_time.tms_stime - start_time.tms_stime) / (double) ticks_per_second;
    double total_cpu_time = user_cpu_time + sys_cpu_time;

    double elapsed_time = (end_time.tms_utime - start_time.tms_utime) / (double) ticks_per_second;

    double cpu_usage = 0.0;
    if (elapsed_time > 0) {
        cpu_usage = (total_cpu_time / elapsed_time) * 100.0;
    }

    struct rusage usage;
    getrusage(RUSAGE_SELF, &usage);
    double ram_usage = usage.ru_maxrss / 1024.0;
    return make_tuple(cpu_usage, ram_usage, psnr_value);
}

tuple<double, double, double, double> process_image(const string& image_path, const string& output_path, int kernel_size) {
    Mat image = imread(image_path, IMREAD_UNCHANGED);

    if (image.empty()) {
        throw runtime_error("Error loading image: " + image_path);
    }

    if (image.rows > 720 || image.cols > 1280) {
        throw runtime_error("Image size exceeds the allowed dimensions of 1280x720p");
    }

    struct tms start_time, end_time;
    clock_t start_clock = times(&start_time);

    Mat filtered_image = apply_median_filter(image, kernel_size);

    clock_t end_clock = times(&end_time);
    double inference_time = (end_clock - start_clock) / (double) CLOCKS_PER_SEC * 1000;

    auto [cpu_usage, ram_usage, psnr_value] = measure_metrics(image, filtered_image, inference_time, start_time, end_time);

    imwrite(output_path, filtered_image);

    return make_tuple(cpu_usage, ram_usage, inference_time, psnr_value);
}

void process_dataset(const string& root_folder, int kernel_size = 3) {
    vector<string> noisy_folders = {"noisy5", "noisy15", "noisy35"};
    vector<string> denoised_folders = {"denoised5", "denoised15", "denoised35"};

    double total_cpu_usage = 0.0;
    double total_ram_usage = 0.0;
    double total_inference_time = 0.0;
    double total_psnr = 0.0;
    int image_count = 0;

    for (size_t i = 0; i < noisy_folders.size(); ++i) {
        string input_folder = root_folder + "/" + noisy_folders[i];
        string output_folder = root_folder + "/" + denoised_folders[i];

        if (!fs::exists(output_folder)) {
            fs::create_directory(output_folder);
        }

        for (const auto& entry : fs::directory_iterator(input_folder)) {
            string image_path = entry.path().string();
            if (image_path.substr(image_path.find_last_of(".") + 1) == "png" ||
                image_path.substr(image_path.find_last_of(".") + 1) == "jpg" ||
                image_path.substr(image_path.find_last_of(".") + 1) == "jpeg") {

                string output_path = output_folder + "/" + entry.path().filename().string();

                auto [cpu_usage, ram_usage, inference_time, psnr_value] = process_image(image_path, output_path, kernel_size);

                total_cpu_usage += cpu_usage;
                total_ram_usage += ram_usage;
                total_inference_time += inference_time;
                total_psnr += psnr_value;
                ++image_count;
            }
        }
    }

    if (image_count > 0) {
        double avg_cpu_usage = total_cpu_usage / image_count;
        double avg_ram_usage = total_ram_usage / image_count;
        double avg_inference_time = total_inference_time / image_count;
        double avg_psnr = total_psnr / image_count;

        cout << "Total CPU Usage (%): " << avg_cpu_usage << endl;
        cout << "Total RAM Usage (MB): " << avg_ram_usage << endl;
        cout << "Total Inference Time (ms): " << avg_inference_time << endl;
        cout << "Average PSNR (dB): " << avg_psnr << endl;
    } else {
        cout << "No images processed." << endl;
    }
}

int main() {
    string root_folder = "/content/drive/MyDrive/dataset";
    process_dataset(root_folder, 3);
    return 0;
}


In [ ]:
!g++ -std=c++17 -o median median.cpp `pkg-config --cflags --libs opencv4`
!./median